# Chroma to Pinecone[[🔗Link](https://python.langchain.com/docs/integrations/vectorstores/pinecone/)]

<span style="color:green;">_**opensource(무료) 중에는 `Faiss`**_ 도 대안으로 생각할 수 있다.</span>

pinecone은 in-memory기반의 db와 다르게 AWS cloud환경에서 db를 구축한다.  
일반적으로 RDB에서의 database를 `인덱스`라고 부른다.  
pinecone에서는 인덱스를 5개 즉, 5개의 db를 무료로 지원한다.  

`pinecone의 api_key`는 chroma와 달리 따로 필요하므로 pinecone console에서 api key를 발급받아야 한다.

pinecone에서 index(DB)를 생성하기 위해서 pinecone console에서 index를 아래와 같이 생성해야 한다.  
embedding과 관련된 model을 선택하는데 openai를 사용하므로 `text-embedding-3-large`를 선택한다.  
여기서 dimension(vector의 길이)과 metric(유사도 측정 방법; 유클리디언과 코사인 유사도)을 선택할 수 있다.  

유사도
- 유클리디언: 내적이므로 값이 `클수록 유사도가 높다.`
- 코사인: 두 vector 사이의 각도를 재는 것으로 값이 `낮을수록 유사도가 높다.`

![create-pinecone-db](./create_pinecone_DB.png)


<span style="color: orange;">
위에서 dimension을 1024로 설정했는데 embedding-large의 dimension은 3072로 오류가 나 index를 다시 삭제하고 다시 생성함.  
</span>

## Embedding Dimension이란?

# ![dimension explanation](./dimension.png)


In [2]:
%pip install -Uq langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv()

pc = Pinecone()
db_name = "tax"

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
  add_start_index=True,
)

splitted_docs = Docx2txtLoader("./tax.docx").load_and_split(text_splitter=text_splitter)

# 아래 코드는 실행할 때마다 document를 통해 raw가 추가되므로 처음 한 번만 실행한다.
# pinecone dashboard를 보면 `browser tab`에서 raw(data)들이 들어온 것을 확인할 수 있다.
# tax_vector_store = PineconeVectorStore.from_documents(
#   documents=splitted_docs,
#   embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
#   index_name=db_name, # image에서 db(index)의 이름을 "tax"로 지정함
# )


# ! 위에서 row(data)를 가지고 있는 pinecone index를 생성하였으므로 기존 index에서 가져오기
tax_pinecone_store = PineconeVectorStore.from_existing_index(
  index_name=db_name,
  embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
)


[Document(id='e534f74b-7425-44ee-acbb-09bd053e23bf', metadata={'source': './tax.docx', 'start_index': 6612.0}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기

# Retrieval Phase

In [8]:
retrieved_docs = tax_pinecone_store.similarity_search("소득세란 무엇인가요?", k = 1)
retrieved_docs

[Document(id='e534f74b-7425-44ee-acbb-09bd053e23bf', metadata={'source': './tax.docx', 'start_index': 6612.0}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기

# QA Chain

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=tax_pinecone_store.as_retriever(kwargs={"k": 1}),
  chain_type_kwargs={"prompt": prompt}
)

/Users/pgj/.pyenv/versions/3.13.3/envs/inflearn-rag-llm-application/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
result = qa_chain.invoke({"query": query})["result"]

result

'죄송하지만, 주어진 문맥에서 연봉 5천만원인 직장인의 소득세 계산에 관한 구체적인 정보를 찾을 수 없습니다. 소득세는 소득 금액, 공제 항목, 세율 등의 여러 요소에 따라 다를 수 있으며, 정확한 계산을 위해 관련 세법과 조항을 참조해 보시기 바랍니다.'

<span style="color:lightgreen;">이렇게 langchain을 활용하면 database가 변경될 때나 RAG를 구축할 때의 loader 등등을 쉽게 깔아끼울 수 있다.</span>